In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path

test_path = Path("/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/")

img_paths = list(test_path.glob("images/*.jpg"))
img_paths.sort()
print(img_paths)

[PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_201.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_202.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_203.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_204.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_205.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_206.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_207.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_208.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_209.jpg'), PosixPath('/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_210.jpg'), PosixPath('/content/drive/MyDrive/FYP C

In [ ]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
from matplotlib import cm as CM


<ipython-input-4-bc5c0a584104>:8: DeprecationWarning: Please import `gaussian_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter


In [ ]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [9]:
for img_path in img_paths:
    img_path=str(img_path)
    print(img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','density_map'), 'w') as hf:
            hf['density'] = k


/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_201.jpg
(530, 800)
generate density...


<ipython-input-5-3a449be3d42b>:24: DeprecationWarning: Please import `gaussian_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')


done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_202.jpg
(530, 800)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_203.jpg
(683, 1024)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_204.jpg
(695, 1024)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_205.jpg
(798, 1017)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_206.jpg
(683, 1024)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_207.jpg
(442, 650)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_208.jpg
(683, 1024)
generate density...
done.
/content/drive/MyDrive/FYP Content/ShanghaiTech/partA/validate/images/IMG_209.jpg
(619, 1000)
generate density...
done.
/content/drive/MyDrive/FYP Content/S